<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/RLHF_instruct_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Supervised Fine-Tuning (SFT): Train a base language model on labeled datasets to perform specific instructions.

In [ ]:
# URL: https://wandb.ai/settings#api
# # api_key: 762475e7d3a86eb2d9ad7cf3860dcbafb1a1448a
# !pip install wandb -qU
# import wandb
# wandb.login()

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Mock dataset: Instruction-Output pairs
dataset = [
    {"instruction": "Translate 'Hello' to Spanish.", "output": "Hola"},
    {"instruction": "What is 2 + 2?", "output": "4"},
    {"instruction": "Write a short poem about the moon.", "output": "The moon glows bright, in the silent night."}
]


# Preprocess dataset
# Set the pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token
def preprocess(data):
    inputs = [f"Instruction: {d['instruction']}\nOutput:" for d in data]
    labels = [f"{d['output']}" for d in data]
    tokenized = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
    tokenized_labels = tokenizer(labels, padding=True, truncation=True, return_tensors="pt").input_ids
    tokenized['labels'] = tokenized_labels
    return tokenized

tokenized_dataset = preprocess(dataset)

# Fine-tuning settings
# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=1,
#     per_device_train_batch_size=2,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_steps=1000,
#     save_total_limit=1
# )
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,  # Save checkpoints every 100 steps
    save_total_limit=1,  # Keep only the latest checkpoint
    save_strategy="epoch"  # Save the model at the end of each epoch
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# trainer.train()

# Save the model and tokenizer manually
trainer.save_model("./results")  # Save the fine-tuned model
tokenizer.save_pretrained("./results")  # Save the tokenizer
# trainer.train()


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./results")
tokenizer = GPT2Tokenizer.from_pretrained("./results")


In [ ]:
# Reward Model Training (RMT): Use human-labeled rankings of model outputs to train a reward model that scores outputs based on their alignment with instructions.

In [ ]:
# import numpy as np

# # Mock reward data: Ranked outputs
# reward_data = [
#     {"instruction": "Translate 'Hello' to Spanish.", "outputs": ["Hola", "Ola", "Hi"], "ranking": [2, 1, 0]},
#     # {"instruction": "What is 2 + 2?", "outputs": ["4", "four", "5"], "ranking": [2, 1, 0]},
# ]

# # # Mock Reward Model: Simple ranking system
# # def reward_model(output, outputs, ranking):
# #     if output in outputs:
# #         index = outputs.index(output)
# #         return ranking[index]
# #     else:
# #         raise ValueError(f"Output '{output}' not found in outputs list.")
# class RewardModel(nn.Module):
#     def __init__(self, base_model_name="gpt2"):
#         super(RewardModel, self).__init__()
#         # Load pre-trained language model
#         self.base_model = AutoModel.from_pretrained(base_model_name)
#         # Add a linear layer for scalar reward prediction
#         self.reward_head = nn.Linear(self.base_model.config.hidden_size, 1)

#     def forward(self, input_ids, attention_mask=None):
#         # Extract hidden states from the base model
#         outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_states = outputs.last_hidden_state
#         # Use the [CLS] token hidden state (or first token for GPT)
#         cls_embedding = hidden_states[:, 0, :]
#         # Predict reward
#         reward = self.reward_head(cls_embedding)
#         return reward



# # Convert outputs to numeric IDs
# reward_dataset = []
# for item in reward_data:
#     for i, output in enumerate(item["outputs"]):
#         reward_dataset.append((item["instruction"], output, item["ranking"][i]))

# # Example for demonstration:
# # In practice, this would involve training a neural network on the reward dataset.


In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer

# Reward Model Definition
class RewardModel(nn.Module):
    def __init__(self, base_model_name="gpt2"):
        super(RewardModel, self).__init__()
        self.base_model = AutoModel.from_pretrained(base_model_name)
        self.reward_head = nn.Linear(self.base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # Use the first token
        reward = self.reward_head(cls_embedding)
        return reward

# Mock reward data: Instructions, outputs, and rankings
reward_data = [
    {"instruction": "Translate 'Hello' to Spanish.", "outputs": ["Hola", "Ola", "Hi"], "ranking": [2, 1, 0]},
    {"instruction": "What is 2 + 2?", "outputs": ["4", "five", "3"], "ranking": [2, 0, 0]},
]

# Prepare reward data
def prepare_reward_data(data, tokenizer):
    inputs, rewards = [], []
    tokenizer.pad_token = tokenizer.eos_token
    for item in data:
        for output, rank in zip(item["outputs"], item["ranking"]):
            input_text = f"Instruction: {item['instruction']}\nOutput: {output}"

            tokenized = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")
            inputs.append(tokenized)
            rewards.append(rank)
    return inputs, torch.tensor(rewards, dtype=torch.float32).unsqueeze(1)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
reward_model = RewardModel(base_model_name="gpt2")
reward_model.train()

inputs, labels = prepare_reward_data(reward_data, tokenizer)

# Train reward model
optimizer = optim.Adam(reward_model.parameters(), lr=5e-5)
loss_fn = nn.MSELoss()
epochs = 3

for epoch in range(epochs):
    total_loss = 0
    for i, input_data in enumerate(inputs):
        input_ids = input_data["input_ids"]
        attention_mask = input_data["attention_mask"]
        optimizer.zero_grad()
        predictions = reward_model(input_ids, attention_mask)
        loss = loss_fn(predictions, labels[i])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 7.8012
Epoch 2, Loss: 11.1984
Epoch 3, Loss: 6.1715


In [ ]:
# Reinforcement Learning (RL): Fine-tune the model using reinforcement learning to optimize the reward model's score.

In [ ]:
# from transformers import pipeline
# from torch.optim import Adam

# # Load the fine-tuned model
# fine_tuned_model = GPT2LMHeadModel.from_pretrained("./results")

# # PPO setup
# optimizer = Adam(fine_tuned_model.parameters(), lr=5e-5)

# # Mock PPO Loop
# for epoch in range(5):  # Simulate 5 epochs of RL
#     for item in reward_data:
#         instruction = item["instruction"]
#         outputs = item["outputs"]
#         ranking = item["ranking"]

#         # Generate new output and compute reward
#         response = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)(f"Instruction: {instruction}\nOutput:")
#         generated_text = response[0]["generated_text"]
#         print(generated_text)
#         # reward = reward_model(generated_text, outputs, ranking)
#         reward = reward_model('Hola', outputs, ranking)

#         # Optimize using the reward
#         loss = -reward  # Negative reward as loss
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

In [ ]:
from transformers import pipeline

# Load fine-tuned model and tokenizer
# policy_model = GPT2LMHeadModel.from_pretrained("./sft_model")
# policy_tokenizer = GPT2Tokenizer.from_pretrained("./sft_model")
# results
policy_model = GPT2LMHeadModel.from_pretrained("./results")
policy_tokenizer = GPT2Tokenizer.from_pretrained("./results")

# Reward Pipeline
reward_model.eval()

# Simplified RLHF loop
optimizer = optim.Adam(policy_model.parameters(), lr=5e-5)

for step in range(10):  # 10 RL steps
    # Mock instruction
    instruction = "Translate 'Hello' to Spanish."
    input_prompt = f"Instruction: {instruction}\nOutput:"

    # Generate response
    response = pipeline("text-generation", model=policy_model, tokenizer=policy_tokenizer)(
        input_prompt, max_length=20, num_return_sequences=1, return_full_text=False
    )
    generated_output = response[0]["generated_text"]

    # Compute reward
    tokenized_input = tokenizer(f"{input_prompt}{generated_output}", return_tensors="pt")
    with torch.no_grad():
        reward = reward_model(tokenized_input["input_ids"]).item()

    # Reinforcement learning: Reward maximization
    loss = -torch.tensor(reward)  # Minimize negative reward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Step {step + 1}, Reward: {reward:.4f}")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn